### How to read this notebook

Parts of this notebook require an active Python server, which is why we recommend it be viewed in [Binder](https://mybinder.org/v2/gh/KrishnaswamyLab/visualization_selection/master?filepath=main_tex.ipynb). You can also run it on your own local Jupyter server.

To advance through the notebook, press Shift+Return to run the code in each cell. Some plots will be missing if you read the notebook without running the code.

In [1]:
import warnings
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scprep
from blog_tools import data, embed, interact
%matplotlib inline

/home/scottgigante/.local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Introduction 

#### What is good visualization?

Despite humans only being capable of interpreting data in three dimensions, real-world data often exists in much higher dimensions, making the task of understanding the structure of such data difficult. As a result, exploring high-dimensional data typically involves some form of dimensionality reduction to two or three dimenisons for visualization. The goal of this technique is to gain insight about the structure of the data, specifically the relationships between points. From these observations, one can generate hypotheses about the dataset. In this way, visualization is an important tool for narrowing the scope of investigation and aids in the selection of tools for future analysis. The importance of this problem is reflected in the many visualization tools have been developed for high-dimensional data. To name a few, we have: [Principal Components Analysis](https://www.sciencedirect.com/science/article/pii/0169743987800849), [Multidimensional Scaling](https://link.springer.com/article/10.1007/BF02289565), [Diffusion Maps](https://www.sciencedirect.com/science/article/pii/S1063520306000546), [Isometric Mapping](https://science.sciencemag.org/content/290/5500/2319.abstract), [Laplcaian Eigenmaps](https://www.semanticscholar.org/paper/Laplacian-Eigenmaps-for-Dimensionality-Reduction-Belkin-Niyogi/0ce8879ea7fc0e96fd0e4e242a46002010f86e18), [Locally Linear Embedding](https://science.sciencemag.org/content/290/5500/2323.full), [t-distributed Stochastic Neighbor Embedding](http://www.jmlr.org/papers/v9/vandermaaten08a.html), [Uniform Manifold Approximation](https://arxiv.org/abs/1802.03426), and [Potential of Heat Affinity Transition Embedding](https://www.biorxiv.org/content/10.1101/120378v4) to name a few. However, along with a diversity of tools comes a diversity of associated biases and assumptions that these methods introduce, and the dreaded problem of selecting the right tool for the job.

Take, for example, t-SNE. At the time of writing, [Visualizing Data using t-SNE (2008)](http://www.jmlr.org/papers/v9/vandermaaten08a.html) has over 8,400 citations. In some fields, like computational biology, it's difficult to find a published paper without a t-SNE visualization. However, t-SNE has a unique set of biases that lend it to misinterpretation; these limitations are explored in the Distill article [How to Use t-SNE Effectively](https://distill.pub/2016/misread-tsne/). Through a set of key examples, the authors show how parameter selection and intrinsic biases in the t-SNE algorithm can lead to misleading visualizations. This article has become a widely referenced teaching tool for the rational application of the method. 

Here, we seek to take the question of *How to Use t-SNE Effectively* and go one step further: How can you select among the many visualization tools effectively? How can you judge the results of each method? What benchmarks should one employ when considering different methods? Which methods are best suited for different data modalities? The remainder of this article will be divided into three sections. 

#### Structure of the article

1. [Selecting toy datasets for comparing methods](#selecting-toy-datasets-for-comparing-methods)

First, we will talk about different kinds of data structures and introduce a few toy datasets that we will use to compare different visualization methods. Next, we will use these datasets to discuss the algorithms behind six popular dimenisonality reduction methods: PCA, MDS, ISOMAP, t-SNE, UMAP, and PHATE.

2. [Parameter selection](#parameter-selection)

We will then discuss sensitivity to parameter choices and methods for quanitfying the accuracy of a visualization method. 

**TODO: quantification?**

3. [Real world data](#real-world-data)

Finally, we will apply our seven comparison methods to large, real-world datasets. The emphasis of this section will be how a particular visualiation tool biases the observer towards a set of conclusions about a given dataset.

#### Dimensionality reduction vs. visualization

Before we go further, it will be useful to clarify the distinction between dimensionality reduction algorithms and visualization algorithms. In fact, visualization algorithms are a subset of dimensionality reduction techniques. The important distiction here is that some dimensionality reduction algorithms are useful for reducing computational complexity of some machine learning tasks, but are not good for creating interpretable two- or three-dimensional visualizations. Take for example, dimensionality reduction by random orthogonal projection, often used in [approximate nearest-neighbors search](https://dl.acm.org/citation.cfm?id=276877). This method take a set of $n$ random orthogonal vectors in the original $d$-dimensional data space with $n << d$. This approach preserves most of the structure of the data so that statistics (such as pairwise distances) can be calculated more quickly. However, random projections evenly preserves information across all $n$ dimensions and will not product features useful for visualizing the relationships between data points. To count as a visualization tool, an algorithm must prioritize information preservation in 2 or 3 dimensions.

## Selecting toy datasets for comparing methods

Rigorous application of any computational technique starts with considering the desired application and how well a given method's biases fits that application. Generally, a good place to start is by creating *toy data*. A good toy data set is small, easy to understand intuitively, and has a clear heuristic for a sucessful visualzation. In *How to Use t-SNE Effectively*, the authors present several compelling toy datasets. Here, we will consider a few of these along with some extras: a graph, small collection of handwritten 7's, and [2000 pictures of Brendan Frey's face](https://cs.nyu.edu/~roweis/data/frey_rawface.jpg).

![Ground truth images of six datasets](img/ground_truth.png)

**TODO: The tree image actually has another branch now. See the PHATE image below**

We selected these datasets because of the varying structures and modalites. 

* **Swiss roll:** This dataset is adapted from `sklearn`'s `dataset.make_swiss_roll()` method. The data exists in three dimensions with one latent dimension: the distance of each point from the center. This is represented by the color of the points in the above plots. The first two dimensions are generated using sines and cosines of this latent dimension and the third dimension is generated as uniform random noise. In this dataset, the noise dimension is slightly larger than the two shown above. As we will see, this proves problematic for some algorithms. The "ideal" visualization of the swiss roll is a single line that represents the latent dimension of that dataset.

  
* **Three blobs:** The three blobs here are gaussian clouds that were first generated in just two dimensions and then linearly projected into 200 dimensions using random Gaussian noise. In the original space, the ratio between the orange and blue blobs is 1/5th the distance between orange and green blobs. The goal here is to preserve the relative distances between the blobs while rotating the data back into two dimensions.

  
* **Uneven circle:** Here, the data points are distributed along a circle in 20 dimensions, but the spacing between the points is irregular. The color of each point represents it's angular position.


* **Digits:** This is the dataset found in `sklearn.datasets.load_digits()` method. According to the User Guide, the data is a copy of the test set of [the UCI ML hand-written digits dataset](https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits). We're just using the 179 sevens. Each image is 8x8 pixels. Because the digits are handwritten, there are natural variations in the digits. For example, some images have a cross-bar and other do not.

  
* **Frey faces:** This dataset consists of 200 images of Brendan Frey's face. In this dataset, Frey is making a series of facial expressions and many intermediate expressions are available between images. Here, we should be able to identify some continuous progressions in facial expression. For example, we should be able to see a smooth transition between a neutral face and a smile or frown.

  
* **Tree:** For our final dataset, we look at a collection of points arranges in a tree, generated using the tool [**Splatter**](https://bioconductor.org/packages/release/bioc/html/splatter.html). This data emulates smooth transitions between various states of biological systems. All of the branches of the tree are of equal length, but the points are not spaced evenly along the branches. Furthermore, some of the features change non-linearly along branches. The goal here is to recreate the topology of the six branches shown above.

  

With these toy datasets in hand, we can begin to compare methods.

### Introducing... _the algorithms_!

For this blog, we wanted to focus on a mix of classic and popular algorithms. Not many people use PCA for rigorous visualization of high-dimensional data (looking at you, [population geneticists](https://blog.insito.me/why-pca-and-genetics-are-a-match-made-in-heaven-6042ea027cf0)), but it is a common tool for preliminary analysis. Similarly, MDS, which is actually a collection of methods, is not as frequently applied for data analysis but serves as a foundation for non-linear dimensionality reduction. On the other hand, PHATE is a relatively new method for visualizing high dimensional data that chose to include because we developed it.

In this section, we will break introduce the algorithms one by one, give a brief overview of how the algorithm works, then show the results of that algorithm on our test cases.


In [2]:
algorithms = embed.__all__

In [7]:
tab_widget = interact.TabWidget()
for algorithm in algorithms:
    name = algorithm.__name__
    with tab_widget.new_tab(algorithm.__name__):
        interact.display_markdown("md/discussion-{}.md".format(name),
                                  placeholder='Introduction to {}'.format(name))

tab_widget.display()

## Parameter selection

In [9]:
import pickle
results = pickle.load(open("data/parameter_search.pickle", 'rb'))
dataset = data.swissroll()

When running a visualization tool, the user is inevitably faced with the daunting choice of various parameters. Some parameters are robust and barely affect the result of the visualization; others drastically affect the results and need to be tuned with care.  

In [10]:
tab_widget = interact.TabWidget()
for algorithm in results.keys():
    with tab_widget.new_tab(algorithm):
        interact.parameter_plot(algorithm, results, dataset.X_true, c=dataset.c)

tab_widget.display()

**TODO: text**

## Real world data

#### The role of real data in comparing visualization algorithms

Toy data is useful because it provides insights into the kinds of idiosyncracies that a given algorithm is sensitive to. However, it is also important that a dimensionality reduction algorithm performs well on real world data. Although you lose access to ground truth, it is still possible to compare algorithms. 

Generally spearking it useful to first consider well existing knowledge is preserved by an given embedding. Are groups of data points that are known to be closely related displayed proximally in the visualization? Can longer-range relationships be observed across the plot?

Once these properties have been confirmed, we usually will generate some hypothesis about the data and further examine trends within the data. This might mean gathering additional information about the data by integrating another outside dataset or performing another experiment. Although this process is challenging, the real utility of a visualization is how successful one can be by making hypotheses from the visualization and testing them using an independent method.

Here, we will describe three real world datasets and compare each visualization algorithm on each dataset. Again, we're going to use default parameters for every method.

### Analysis of population genetics data

#### The 1000 Genomes Project

The [1000 Genomes Project](http://www.internationalgenome.org/) is an international effort to understand genetic variation across human populations. Originating in 2007, the project completed in 2013 describing variation in roughly 3000 individuals from 26 populations across more than 80,000 genetics loci. The project is a high quality resource for understanding differences between people from different regions across the globe.

This data is useful for visualization because we have an intuitive understanding of geographic relationships between people, but the data is so high dimensional that understanding if these structures are preserved in the genetic space is infeasible.

In the 1000 Genomes data we also have access to detailed population information about each individual in the study. We expect that geographically proximal populations are genotypically similar and should therefore be grouped together in the visualization.

#### What does the data look like?

As mentioned above, the 1000 Genomes dataset contains genetic information about 3000 individuals. The data we will embed is called a genotype matrix. The rows of this matrix correspond to each individual in the study, and each column corresponds to a [Single Nucleotide Polymorphism (SNP)](https://ghr.nlm.nih.gov/primer/genomicresearch/snp). Each entry in the matrix is `0` if the individual is homozygous for the reference allele, `1` if the individual is heterozygous for the reference and alternative alleles, and `2` if the individual is homozygous for the alternative alleles. If these terms are unfamiliar to you, we suggest reading the NIH primer on SNPs. All that's really important to know if that each entry in the matrix indicates a specific DNA element of each individual in the dataset.

Thankfully Alex Diaz-Papkovich from Simon Gravel's lab at McGill provides [scripts to load and preprocess this data on Github](https://github.com/diazale/1KGP_dimred). All we need to do is run his scripts through the algorithms for our comparison.



#### Comparing visualization algorithms on the 1000 Genomes data.

![1000 - Genomes comparison](./img/1000_genomes.comparison.2x3.png)

This data is complex with many different populations spanning six continents. Generally speaking, Blue corresponds to East Asian, Green is South Asain, Yellow/Orange is African, Red represents European, and Pink is Ad Mixed American. The exact breakdown of each population code is given by the following table.

| Population Code  | Population Description                                             | Super Population  | 
|------------------|--------------------------------------------------------------------|------------------------| 
| CHB              | Han Chinese in Beijing, China                                      | East Asian             | 
| JPT              | Japanese in Tokyo, Japan                                           | East Asian             | 
| CHS              | Southern Han Chinese                                               | East Asian             | 
| CDX              | Chinese Dai in Xishuangbanna, China                                | East Asian             | 
| KHV              | Kinh in Ho Chi Minh City, Vietnam                                  | East Asian             | 
| CEU              | Utah Residents (CEPH) with Northern and Western European Ancestry  | European               | 
| TSI              | Toscani in Italia                                                  | European               | 
| FIN              | Finnish in Finland                                                 | European               | 
| GBR              | British in England and Scotland                                    | European               | 
| IBS              | Iberian Population in Spain                                        | European               | 
| YRI              | Yoruba in Ibadan, Nigeria                                          | African                | 
| LWK              | Luhya in Webuye, Kenya                                             | African                | 
| GWD              | Gambian in Western Divisions in the Gambia                         | African                | 
| MSL              | Mende in Sierra Leone                                              | African                | 
| ESN              | Esan in Nigeria                                                    | African                | 
| ASW              | Americans of African Ancestry in SW USA                            | African                | 
| ACB              | African Caribbeans in Barbados                                     | African                | 
| MXL              | Mexican Ancestry from Los Angeles USA                              | Ad Mixed American      | 
| PUR              | Puerto Ricans from Puerto Rico                                     | Ad Mixed American      | 
| CLM              | Colombians from Medellin, Colombia                                 | Ad Mixed American      | 
| PEL              | Peruvians from Lima, Peru                                          | Ad Mixed American      | 
| GIH              | Gujarati Indian from Houston, Texas                                | South Asian            | 
| PJL              | Punjabi from Lahore, Pakistan                                      | South Asian            | 
| BEB              | Bengali from Bangladesh                                            | South Asian            | 
| STU              | Sri Lankan Tamil from the UK                                       | South Asian            | 
| ITU              | Indian Telugu from the UK                                          | South Asian            | 


#### What do we see?

**PCA** is the most common method for visualizaing population genetics information, but limitations in this method mean that as larger datasets become available, researchers are often searching for other tools [Diaz-Papkovich et al. (2019)](https://www.biorxiv.org/content/10.1101/423632v2). Examining the output of PCA on this dataset, we can see clear separation between the African populations denoted by the Yellow/Orange points from the rest of the populations. Similarly, the East Asian population is separated from the rest. However, it is difficult to make statements about the American, South Asian, or European populations.

**MDS** has some artistic appeal here as the visualization resembles a globe. However, we have a problem here of the pink Ad Mixed American population divided in two by the South Asian population. We see this as well with PCA, but this does not reconcile with our understanding of genetic variation between these populations. Generally speaking, following humanity's migration out of Africa has been characterized by increasing genetic isolation with limited examples of population mixing ([Hellenthal et al. 2014.](https://www.ncbi.nlm.nih.gov/pubmed/24531965/); [Norris et al. 2018.](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6288849/)).

**ISOMAP** performs poorly here. See how the South Asian and East Asian populations are completely reduced to a very small region plot. This poorly represents the diversity of those populations.

**TSNE** can be very sensitive to outliers. Here, many individuals have essentially no neighbors in the high dimensional space and therefore they get evenly distributed around the visualization. Not so useful.

**UMAP** was shown to work well for population genetics data by [Diaz-Papkovich et al. (2019)](https://www.biorxiv.org/content/10.1101/423632v2), and this was actually the inspriration for this real-world data application. There, the authors used some parameter tuning to generate the plots in the manuscript, so their visualization is slightly nicer than what we see here. However, the game of this blog post is to use default parameters in all comparisons. We see here that the defaults for UMAP produce a visualization that is very compressed, making it hard to see fine-grain relationships between points.

**PHATE** does a good job here of presenting each population as a separate group of points on the plots. The default parameters also compress a lot of the variation into a few smooth trajectories. One of the places where PHATE shines here is when you consider the way that orderings of each principle component are preserved within each group of points.

#### Examining PCS along each visualization

![1000 - Genomes comparison](./img/1000_genomes.comparison.PC1.png)

![1000 - Genomes comparison](./img/1000_genomes.comparison.PC2.png)

![1000 - Genomes comparison](./img/1000_genomes.comparison.PC3.png)

In the above plots, we're looking at the first three principal components loadings on each individual in the dataset. If you look at PC1 in PHATE vs UMAP, PHATE preserved the ordering of cells by increasing PC1 loadings whereas UMAP splits the data points with high PC1 coordinates across two different clusters. Examining PC2 and PC3, we see that the embedding produced by MDS and TSNE order points non-monotonically by their coordinate loadings. This is a little frustrating because we generally think of these coordinates as representing axes of diversity throughout each population.

### Analysis of single cell RNA-sequencing data


#### Retinal biopolar scRNA-seq data

